<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation (RAG) Baseline
## Overview  
In this time, we will check the baseline code.
The goal of this project is to provide students with hands-on experience in handling and enhancing Large Language Models (LLMs) provided by [**Upstage**](https://www.upstage.ai) (Solar).

You can use any engineering method for improving benchmark performance excluding direct training (Fine-tuning).

*Collecting data directly related to the test set is considered cheating e.g., using MMLU-pro dataset or EWHA.pdf for KB*

# Baseline

In [ ]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

In [ ]:
# Additional Contents : https://wikidocs.net/253106

In [ ]:
# set parameters

api_key = "YOUR KEYS"
data_path = "YOUR PATH" # folder path containing ewah.pdf and samples.csv

In [ ]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os
 
UPSTAGE_API_KEY = api_key

layzer = UpstageLayoutAnalysisLoader(api_key=UPSTAGE_API_KEY,file_path=os.path.join(data_path, 'ewha.pdf'), output_type="text")

docs = layzer.load()  # or layzer.lazy_load()

/usr/local/lib/python3.10/dist-packages/langchain_upstage/layout_analysis.py:135: UserWarning: UpstageLayoutAnalysisLoader is deprecated.Please use langchain_upstage.document_parse.UpstageDocumentParseLoader instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_upstage/layout_analysis_parsers.py:160: UserWarning: UpstageLayoutAnalysisParser is deprecated.Please use langchain_upstage.document_parse_parsers.UpstageDocumentParseParser instead.
  warnings.warn(


In [ ]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))


Splits: 53


In [ ]:
print(splits[0])

page_content='이화여자대학교 학칙1946. 8. 15. 제정
2017. 8. 16. 개정제1장 총칙제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.제2조(명칭) 본교는 이화여자대학교라 부른다.
제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)제2장 편제제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보
건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).
(개정 2016.6.16., 2017.5.15.)
[전문개정 2015.11.27.]제5조(학부․학과․전공 및 정원) ① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표
1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.15.)② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집
할 수 있다. (개정 1999.2.9., 2017.5.15.)
③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위
로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후에 총장의 승인을 얻어 이수할
전공을 결정하게 하되 이에 필요한 사항은 총장이 따로 정한다

In [ ]:
# read samples.csv file

import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [ ]:
prompts, answers = read_data(os.path.join(data_path, 'samples.csv'))

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = UPSTAGE_API_KEY)

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context.
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm

responses = []

for prompt in prompts:
    response = chain.invoke({"question": prompt, "context": splits[0]})
    responses.append(response.content)

ModuleNotFoundError: No module named 'langchain_upstage'

In [ ]:
for i in responses:
    print(i)
    print('-'*10)


이화여자대학교 학칙에는 영어 및 정보 등에 관한 일정한 기준의 능력이나 자격을 취득한 경우 인정받는 학점에 대한 정보가 없습니다. 따라서, 주어진 선택지 중 어느 것도 정답이 아닙니다.
---------
이화여자대학교 학칙에 대한 내용이며, LMS 시스템의 초기 비밀번호에 대한 정보는 제공되지 않습니다.
---------
The information is not present in the context.
---------
The information is not present in the context.
---------
문맥에서 답변이 없습니다.
---------
문맥에서 질문에 대한 답은 제시되어 있지 않습니다.
---------
The information is not present in the context.
---------
해당 문맥에서는 법의 기능에 대한 정보가 제공되지 않았습니다.
---------
The information is not present in the context.
---------
The information is not present in the context.
---------


In [ ]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [ ]:
# print accuracy

cnt = 0

for answer, response in zip(answers, responses):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")


    if generated_answer == None:
        continue
    if generated_answer in answer:
        cnt += 1

print()
print(f"acc: {(cnt/len(answers))*100}%")

-----
이화여자대학교 학칙에는 영어 및 정보 등에 관한 일정한 기준의 능력이나 자격을 취득한 경우 인정받는 학점에 대한 정보가 없습니다. 따라서, 주어진 선택지 중 어느 것도 정답이 아닙니다.
extraction fail
-----
이화여자대학교 학칙에 대한 내용이며, LMS 시스템의 초기 비밀번호에 대한 정보는 제공되지 않습니다.
extraction fail
-----
The information is not present in the context.
extraction fail
-----
The information is not present in the context.
extraction fail
-----
문맥에서 답변이 없습니다.
extraction fail
-----
문맥에서 질문에 대한 답은 제시되어 있지 않습니다.
extraction fail
-----
The information is not present in the context.
extraction fail
-----
해당 문맥에서는 법의 기능에 대한 정보가 제공되지 않았습니다.
extraction fail
-----
The information is not present in the context.
extraction fail
-----
The information is not present in the context.
extraction fail

acc: 0.0%


### 내가 한 거 추가

In [ ]:
# Only 3 (similarity)

import numpy as np

chunk = []
for index in range(0,len(splits)) : # context 받아오기
    chunk.append(splits[index].page_content)

for prompt in prompts[:5] : # 질문 받아오기 
    embedded_query = query_embeddings.embed_query(prompt.split('\n')[0])
    embedded_documents = passage_embeddings.embed_documents(chunk)

    # 질문(embedded_query):
    similarity = np.array(embedded_query) @ np.array(embedded_documents).T

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

    # 결과 출력
    print(f"\n[Query] {prompt[:10]}")
    print("-" * 10)
    for i, idx in enumerate(sorted_idx):
        print(f"[{i}] 유사도: {similarity[idx]:.3f} | 해당 context: {idx}")